<a href="https://colab.research.google.com/github/seoyeon7/ECG_Data_Analysis/blob/main/Data_analysis_Statistics_for_Filtering_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
def main_start(file_name,plus,num):
  file_full = dataset_path + '/' + file_name
  df = pd.read_csv(file_full, skiprows=[1, 2])

  data=df['signal'].str.strip("[")
  data2=data.str.strip("]")
  dataset = data2.str.split(',', expand=True)

  dataset.columns = ['before', 'after']
  dataset["after"]=dataset["after"].astype('float')
  dataset["before"]=dataset["before"].astype('float')

  dataset = dataset.drop(['before'], axis=1)

  x_des =dataset.describe()
  data1 = x_des.round(5)

  feature_data = pd.DataFrame(columns = ['mean', 'std', 'min', 'max'])
  feature_data.loc[0] = [data1.iloc[1]['after'], data1.iloc[2]['after'], data1.iloc[3]['after'], data1.iloc[7]['after']]

  return feature_data

In [4]:
if __name__=="__main__":
  num_lis = [18, 20, 23, 24, 25, 26, 27, 28, 29, 31, 33]
  dataset=pd.DataFrame(columns = ['mean', 'std', 'min', 'max'])

  for nl in num_lis:
    temp = nl
    dataset_path = '/content/drive/MyDrive/weka/ECG(electrocardiogram) (1)/{}'.format(temp)
    last_dataset=pd.DataFrame(columns = ['mean', 'std', 'min', 'max'])

    if not os.path.isdir(dataset_path):
      exit()
    else:
      file_list = os.listdir(dataset_path)
      for fl in range(len(file_list)):
        data_name = file_list[fl]
        name2=data_name[9:-10]
        last_dataset = last_dataset.append(main_start(data_name,round(10010/temp),temp), ignore_index=True)
        last_dataset.loc[fl, 'classfi'] = name2
      dataset=dataset.append(last_dataset)

In [7]:
dataset

,mean,std,min,max,classfi
0,0.00011,0.10559,-0.235,0.485,Person_09
0,0.00093,0.13777,-0.385,0.490,Person_09
1,0.00085,0.12289,-0.340,0.465,Person_09
0,-0.00101,0.12337,-0.175,0.845,Person_03
1,-0.00165,0.11929,-0.205,0.810,Person_03
0,-0.00025,0.09650,-0.315,0.470,Person_10
1,-0.00084,0.14589,-0.250,1.075,Person_03
2,0.00113,0.10974,-0.240,0.710,Person_01
0,0.00131,0.12475,-0.475,0.630,Person_09
1,-0.00073,0.09755,-0.360,0.410,Person_09


In [5]:
def pandas2arff(df,filename,wekaname = "pandasdata",cleanstringdata=True,cleannan=True):
    """
    converts the pandas dataframe to a weka compatible file
    df: dataframe in pandas format
    filename: the filename you want the weka compatible file to be in
    wekaname: the name you want to give to the weka dataset (this will be visible to you when you open it in Weka)
    cleanstringdata: clean up data which may have spaces and replace with "_", special characters etc which seem to annoy Weka. 
                     To suppress this, set this to False
    cleannan: replaces all nan values with "?" which is Weka's standard for missing values. 
              To suppress this, set this to False
    """
    import re
    
    def cleanstring(s):
        if s!="?":
            return re.sub('[^A-Za-z0-9]+', "_", str(s))
        else:
            return "?"
            
    dfcopy = df #all cleaning operations get done on this copy

    
    if cleannan!=False:
        dfcopy = dfcopy.fillna(-999999999) #this is so that we can swap this out for "?"
        #this makes sure that certain numerical columns with missing values don't get stuck with "object" type
 
    f = open(filename,"w")
    arffList = []
    arffList.append("@relation " + wekaname + "\n")
    #look at each column's dtype. If it's an "object", make it "nominal" under Weka for now (can be changed in source for dates.. etc)
    for i in range(df.shape[1]):
        if dfcopy.dtypes[i]=='O' or (df.columns[i] in ["Class","CLASS","class"]):
            if cleannan!=False:
                dfcopy.iloc[:,i] = dfcopy.iloc[:,i].replace(to_replace=-999999999, value="?")
            if cleanstringdata!=False:
                dfcopy.iloc[:,i] = dfcopy.iloc[:,i].apply(cleanstring)
            _uniqueNominalVals = [str(_i) for _i in np.unique(dfcopy.iloc[:,i])]
            _uniqueNominalVals = ",".join(_uniqueNominalVals)
            _uniqueNominalVals = _uniqueNominalVals.replace("[","")
            _uniqueNominalVals = _uniqueNominalVals.replace("]","")
            _uniqueValuesString = "{" + _uniqueNominalVals +"}" 
            arffList.append("@attribute " + df.columns[i] + _uniqueValuesString + "\n")
        else:
            arffList.append("@attribute " + df.columns[i] + " real\n") 
            #even if it is an integer, let's just deal with it as a real number for now
    arffList.append("@data\n")           
    for i in range(dfcopy.shape[0]):#instances
        _instanceString = ""
        for j in range(df.shape[1]):#features
                if dfcopy.dtypes[j]=='O':
                    _instanceString+="\"" + str(dfcopy.iloc[i,j]) + "\""
                else:
                    _instanceString+=str(dfcopy.iloc[i,j])
                if j!=dfcopy.shape[1]-1:#if it's not the last feature, add a comma
                    _instanceString+=","
        _instanceString+="\n"
        if cleannan!=False:
            _instanceString = _instanceString.replace("-999999999.0","?") #for numeric missing values
            _instanceString = _instanceString.replace("\"?\"","?") #for categorical missing values
        arffList.append(_instanceString)
    f.writelines(arffList)
    f.close()
    del dfcopy
    return True

In [6]:
pandas2arff(dataset,"/content/drive/MyDrive/weka/data_after.arff")

True